In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

# sagemaker: AWS SDK for training and deploying machine learning models.
# boto3: AWS SDK for Python to manage AWS services like S3, EC2, and SageMaker.
# get_image_uri: A utility function to get the container URI for built-in SageMaker algorithms.
# s3_input: A class for specifying S3 locations for training/testing data.
# Session: Manages interactions with SageMaker.

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/24/25 10:53:43] INFO     Found credentials from IAM Role:                                   ]8;id=331099;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=732252;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# bucket_name: Placeholder for your S3 bucket name. It should be unique.
# my_region: Captures the current AWS region of the instance using boto3.
# print(my_region): Displays the region to verify.
# session: Creates a Boto3 session for the specified region.
# sagemaker_client: Creates a client for interacting with SageMaker resources in the given region.

bucket_name = 'iriiss' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)
session = boto3.Session(region_name=my_region)
sagemaker_client = session.client("sagemaker", region_name=my_region)

ap-northeast-1


[01/24/25 10:53:44] INFO     Found credentials from IAM Role:                                   ]8;id=70592;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=103663;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [3]:
import boto3

s3 = boto3.client('s3', region_name=my_region)  # Use `client` instead of `resource` for bucket creation.

try:
    if my_region == 'us-east-1':
        # No LocationConstraint for us-east-1
        s3.create_bucket(Bucket=bucket_name)
    else:
        # Add LocationConstraint for other regions
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': my_region}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error:', e)


                    INFO     Found credentials from IAM Role:                                   ]8;id=45876;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=744722;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

S3 error: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)


s3://iriiss/xgboost-as-a-built-in-algo/output


In [5]:

import pandas as pd
import urllib
from sklearn.model_selection import train_test_split
columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth', 'class']
try:
    urllib.request.urlretrieve ("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", "iris.csv")
    print('Success: downloaded iris.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./iris.csv',header=None, names=columns)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
     
### Train Test split
X = model_data.drop('class', axis=1)  # Features (dropping 'class')
y = model_data['class']  # Target variable (class)



import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape,y_train.shape,y_test.shape)


Success: downloaded iris.csv.
Success: Data loaded into dataframe.
(105, 4) (45, 4) (105,) (45,)


In [6]:
model_data

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
### Saving Train And Test Into Buckets
## We start with Train Data

from sagemaker.inputs import TrainingInput
import os
pd.concat([X_train, y_train],axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
     


[01/24/25 10:53:46] INFO     Found credentials from IAM Role:                                   ]8;id=88320;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=276344;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [8]:
# Test Data Into Buckets
pd.concat([y_train,y_test], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')
     


                    INFO     Found credentials from IAM Role:                                   ]8;id=289893;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=382868;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [9]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
# get_image_uri retrieves the Docker image for XGBoost.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')


                    WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=627175;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=169707;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Defaulting to only available Python version: py3                     ]8;id=421465;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=566321;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#608\608]8;;\

[01/24/25 10:53:47] INFO     Defaulting to only supported image scope: cpu.                       ]8;id=803757;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=384958;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#532\532]8;;\

In [10]:

# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }


In [11]:

# boto3: AWS SDK for Python, used to interact with AWS services like SageMaker and S3.
# sagemaker.Session: Manages interactions with Amazon SageMaker, such as setting up sessions, managing training jobs, and deploying models.
# sagemaker.estimator.Estimator: High-level class to define and manage training jobs, including specifying input data, algorithms, hyperparameters, and compute resources.


import boto3
from sagemaker import Session
from sagemaker.estimator import Estimator

# Create a boto3 session with a valid region name
boto_session = boto3.Session(region_name="ap-northeast-1")

# Initialize the SageMaker session
sagemaker_session = Session(boto_session=boto_session)


# Get the SageMaker role ARN (ensure this is correctly configured in your environment)
role = sagemaker_session.get_caller_identity_arn()

# Define the estimator
estimator = Estimator(
    image_uri=container,
    hyperparameters=hyperparameters,
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    volume_size=5,  # 5 GB
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600,
    sagemaker_session=sagemaker_session  # Pass the session explicitly
)


                    INFO     Found credentials from IAM Role:                                   ]8;id=316083;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=770540;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [ ]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})


                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=55534;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=399108;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=892385;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=733504;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-01-24-10-53-47-660                                             

2025-01-24 10:53:48 Starting - Starting the training job...

In [ ]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')


# Prediction of the Test Data


In [ ]:
# from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
# xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)


In [ ]:
predictions_array


In [ ]:
# The model's performance is evaluated using a confusion matrix:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

# It compares the predicted results with the actual values, showing how well the model performed (e.g., accuracy, false positives, false negatives).



# Deleting end points

In [ ]:
# This removes the live endpoint and deletes all the files from the S3 bucket to avoid unnecessary storage costs.
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()